In [ ]:
# https://github.com/timothyrubin/python_gclda.git

## OpenCL

In [1]:
import pyopencl as cl
platforms = cl.get_platforms()

## Pystan simple example (bernoulli)

In [1]:
import pystan
import numpy as np

In [8]:
## bernoulli distribution
from pystan import stan
# bernoulli model

model_code = """
data {
  int<lower=0> N;
  int<lower=0,upper=1> y[N];
}
parameters {
  real<lower=0,upper=1> theta;
}
model {
  theta ~ beta(0.5, 0.5); // Jeffreys' prior
  for (n in 1:N)
    y[n] ~ bernoulli(theta);
    }
"""   


from pystan import StanModel
data = dict(N=10, y=[0, 1, 0, 1, 0, 1, 0, 1, 1, 1])
sm = StanModel(model_code=model_code)
fit = sm.sampling(data=data)
print(fit)
# reuse model with new data
new_data = dict(N=6, y=[0, 0, 0, 0, 0, 1])
fit2 = sm.sampling(data=new_data)
print(fit2)

INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_d601e865f1676ee91056b0611e70c7be NOW.


Inference for Stan model: anon_model_d601e865f1676ee91056b0611e70c7be.
4 chains, each with iter=2000; warmup=1000; thin=1; 
post-warmup draws per chain=1000, total post-warmup draws=4000.

        mean se_mean     sd   2.5%    25%    50%    75%  97.5%  n_eff   Rhat
theta   0.59  3.7e-3   0.14   0.31    0.5    0.6   0.69   0.85   1469    1.0
lp__   -7.96    0.02   0.77 -10.13  -8.11  -7.67   -7.5  -7.44   1567    1.0

Samples were drawn using NUTS at Mon Nov  4 15:39:25 2019.
For each parameter, n_eff is a crude measure of effective sample size,
and Rhat is the potential scale reduction factor on split chains (at 
convergence, Rhat=1).
Inference for Stan model: anon_model_d601e865f1676ee91056b0611e70c7be.
4 chains, each with iter=2000; warmup=1000; thin=1; 
post-warmup draws per chain=1000, total post-warmup draws=4000.

        mean se_mean     sd   2.5%    25%    50%    75%  97.5%  n_eff   Rhat
theta   0.21  3.9e-3   0.14   0.02    0.1   0.19    0.3   0.55   1360    1.0
lp__   -4.17  

## LDA on Diagnosis Table

In [2]:
## Basic LDA for documents
"""
data {
  int<lower=2> K;               // num topics
  int<lower=2> V;               // num words(unique words, vocabulary)
  int<lower=1> M;               // num docs
  int<lower=1> N;               // total word instances (accumulated lengths of all documents)
  int<lower=1,upper=V> w[N];    // word n
  int<lower=1,upper=M> doc[N];  // doc ID for word n
  vector<lower=0>[K] alpha;     // topic prior
  vector<lower=0>[V] beta;      // word prior
}
parameters {
  simplex[K] theta[M];   // topic dist for doc m
  simplex[V] phi[K];     // word dist for topic k
}
model {
  for (m in 1:M)
    theta[m] ~ dirichlet(alpha);  // prior
  for (k in 1:K)
    phi[k] ~ dirichlet(beta);     // prior
  for (n in 1:N) {
    real gamma[K];
    for (k in 1:K)
      gamma[k] = log(theta[doc[n], k]) + log(phi[k, w[n]]);
    target += log_sum_exp(gamma);  // likelihood;
  }
}
"""

INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_3aaa1aff3be33470f8a5bfa56085d51c NOW.


OrderedDict([('mu', array(-0.10765848))])

In [2]:
## LDA for drug table
ldacode = """
data {
  int<lower=2> K;               // num topics
  int<lower=1> V;               // num drugs
  int<lower=1> M;               // num patients
  int<lower=0,upper=1> w[V,M];    // disease matrix, column major order
  vector<lower=0>[K] beta;     // topic prior
  real alpha1;      // drug prior 1
  real alpha2;      // drug prior 2
}
parameters {
  simplex[K] theta[M];   // topic dist for doc m
  simplex[V] phi[K];     // drug dist for topic k
}
model {
  for (m in 1:M)
    theta[m] ~ dirichlet(beta);  // prior
  for (k in 1:K)
    phi[k] ~ beta(alpha1, alpha2);     // prior
  for (m in 1:M) {
      for (v in 1:V){
          real gamma[K];
          for (k in 1:K)
              gamma[k] = log(theta[m, k]) + log(phi[k, v]);
          target += log_sum_exp(gamma);  // likelihood;
      }
    
  }
}
"""


## read mimic3 data

In [3]:
import numpy as np
import pandas as pd
import re
from os.path import join

from tools import *

from sklearn.decomposition import LatentDirichletAllocation
from scipy.spatial import distance

import matplotlib.pyplot as plt
import numpy as np

## rename EXTRACT_FEATURES to DIMENSION_REDUCTION
read_prefix = "/data/MIMIC3/"
write_prefix = "/data/liu/LDA"
res_prefix = "/data/liu/LDA/lda_result"
res_r_prefix = "/data/liu/LDA/lda_R_result/"

In [4]:
diag_matrix = read_data(join(write_prefix, "diag_matrix_filtered"))
# pres_matrix = read_data(join(write_prefix,"pres_matrix_filtered"))

In [5]:
diag_matrix.head()

,78061,78062,78065,78079,78093,78097,00845,7810,7812,7813,...,77989,78001,78009,7802,78039,7804,78052,78057,7806,78060
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [6]:
diag_matrix.shape

(39356, 1309)

In [7]:
## save or reload model cache
# import pickle

# def save(model, modelname):
#     """Save compiled models for reuse."""
#     with open("%s.pkl"%modelname, "wb") as f:
#         pickle.dump(model,f)
#         # or with a list
#         # pickle.dump([model, fit], f, protocol=-1)

# def load(modelname):
#     """Reload compiled models for reuse."""
#     model = pickle.load(open('%s.pkl')%modelname, 'rb')
#     return model

In [9]:
##Automatically reusing models
from hashlib import md5
def StanModel_cache(model_code, model_name=None, **kwargs):
    """Use just as you would `stan`"""
    code_hash = md5(model_code.encode('ascii')).hexdigest()
    if model_name is None:
        cache_fn = 'cached-model-{}.pkl'.format(code_hash)
    else:
        cache_fn = 'cached-{}-{}.pkl'.format(model_name, code_hash)
    try:
        sm = pickle.load(open(join("cache",cache_fn), 'rb'))
    except:
        sm = pystan.StanModel(model_code=model_code)
        with open(join("cache",cache_fn), 'wb') as f:
            pickle.dump(sm, f)
    else:
        print("Using cached StanModel")
    return sm

In [8]:
# with open("basic_lda_model.pkl" ,"rb") as f:
#     data_dict = pickle.load(f)

In [10]:
#   int<lower=2> K;               // num topics
#   int<lower=1> V;               // num drugs
#   int<lower=1> M;               // num patients
#   int<lower=0,upper=1> w[M,V];    // disease
#   vector<lower=0>[K] beta;     // topic prior
#   real alpha1, alpha2;      // drug prior

def run_model(ldacode,num_topics,iters,df,beta,alpha1,alpha2,model_file="basic_lda_model"):
    
    dat = {
        'K': num_topics,
        'V': df.shape[0],
        'M': df.shape[1],
        'w': df,
        'beta': np.array(num_topics*[beta]),
        'alpha1': alpha1,
        'alpha2': alpha2
    }
    
    sm = StanModel_cache(model_code=ldacode)
    fit = sm.sampling(data=dat, iter=iters, chains=4,seed=2019)
#     save(sm,'basic_lda_diag')
    return fit

In [11]:
test_data = diag_matrix.astype(int)

In [ ]:
fit = run_model(ldacode,5,700,test_data.T,0.01,0.01,0.01)

INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_c6c36827b439bb758be0f697b53ecc1e NOW.


In [ ]:
print(fit)

In [ ]:
trace = fit.extract('theta','phi')


In [ ]:
write2file(pandas.DataFrame(trace['theta'][699]),join(write_prefix,"pystan_diag_matrix_theta"))
write2file(pandas.DataFrame(trace['phi'][699]),join(write_prefix,"pystan_diag_matrix_phi"))

In [ ]:
# model = pystan.StanModel(model_code=ldacode)
# save(model, fit,'basic_lda_model.pkl')